# Tarea Práctica de Statistical Learning I
### Rocío Serguera 19012424

La tarea práctica consiste en usar el mismo dataset del proyecto del curso pasado y e implementar gradient descent aplicado a regresión con este(usando la variable con mayor correlación elegida en el proyecto en anterior(solo una de las 2 del proyecto)),si no terminaron el proyecto anterior no se preocupen, no hay que hacer todo el proyecto, solo el entrenamiento con gradient descent pero usando tensorflow y usar tensorboard para monitorear que el error disminuya conforme avanzan las iteraciones "en vivo"(en lugar de hacerlo con matplotlib hasta el final como lo hicimos antes).
Pueden usar el siguiente ejemplo de regresión con tensorflow: https://www.geeksforgeeks.org/linear-regression-using-tensorflow/ o el notebook usado en clase.

Para la parte de Tensorboard, este nos permite visualizar mucha información relevante, pero nos centraremos en 2 cosas :

La curva de aprendizaje(disminución de error MSE en el tiempo)(en la la pestaña "scalars")
El grafo que representa a nuestro modelo (en la pestaña "graphs")

Requisitos:

- Utilizar solo operaciones vectorizadas.
- Usar BATCH gradient descent (utilizar todo el dataset en cada paso de gradient descent)
- Utilizar tensorboard:
- Agregar el grafo del modelo.
- Usar operaciones con nombre y usar name_spaces para mejorar el orden.
- Agregar un summary.scalar para monitorear el MSE.
- Aplicar reutilización de código: por ejemplo usar programación orientada a objetos para crear el modelo y/o definir el entrenamiento dentro de funciones de python.
- Utilizar las funciones reutilizables de entrenamiento para ejecutar al menos 10 experimentos (variable numero de iteraciones, y diferente learning rate)
- definir un string de configuración para cada experimento.
- crear un directorio para cada experimento basado en el string de configuración.
- utilizar un enfoque investigación científica: antes del experimento formular una hipótesis del comportamiento experado, ejecutar el experimento y utilizar los resultados de tensorboard para conculir.
- basado en los resultados experimentales obtenidos con tensorboard crear una conclusión general y elegir el mejor experimento.
- agregar el contenido de tensorboard al notebook.
- subir el notebook a github y agregar al GES el link
- no utilizar tf.train.GradientDescentOptimizer

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [76]:
%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [2]:
house_prices = np.load("/users/rocioserguera/Desktop/Maestría/Python/1er Trimestre/proyecto_training_data.npy")

In [3]:
# Separando la base de datos en dos datasets

split = int(0.8*len(house_prices))
train, test = house_prices[:split, :], house_prices[split:, :]
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)
# Nombrando columnas df_train
df_train.columns = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsabvGrd', 'YearBuilt', 'LotFrontage']
# Nombrando columnas df_test
df_test.columns = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsabvGrd', 'YearBuilt', 'LotFrontage']

x_train = df_train['OverallQual']
y_train = df_train['SalePrice']

In [44]:

# Mean Squared Error Cost Function
def error(y,y_pred):
    return 1/2*tf.reduce_mean(tf.math.square(y - y_pred))

#Gradient Descent Function
def gd(y,y_pred,m,b):
    return tf.gradients(error(y,y_pred),[m,b])

# Initializing variables m and b
m = tf.Variable(np.random.randn(), name = "m") 
b = tf.Variable(np.random.randn(), name = "b")

def run_training(x, y, epochs, lr, print_error_every=0):
    
    x = tf.placeholder(tf.float32,[len(x)],"x")
    y = tf.placeholder(tf.float32,[len(y)],"y")
    # Hypothesis 
    y_pred = tf.add(tf.multiply(x, m), b) 
    
    # Calculating Gradient Descent 
    gradient = gd(y,y_pred,m,b)
    
    # Global Variables Initializer 
    init = tf.global_variables_initializer() 
    
    # Saving data 
    m_ = tf.assign(m, m - lr*gradient[0] )
    b_ = tf.assign(b, b - lr*gradient[1] )
    parameters = tf.group(m_,b_)
    
    error_= error(y,y_pred)
    error_scalar = tf.summary.scalar('Error', error_)
    # Starting the Tensorflow Session 
    with tf.train.MonitoredSession() as sess: 
        # Initializing the Variables 
        sess.run(init)
        feed_dict = {x: x_train, y:y_train}
        writer = tf.summary.FileWriter('/users/rocioserguera/Desktop/Tensorboard', sess.graph)
        # Iterating through all the epochs 
        for epoch in range(epochs):
            sess.run(gradient,feed_dict=feed_dict)
            sess.run(parameters,feed_dict=feed_dict)
            summary = sess.run(error_scalar,feed_dict)
            writer.add_summary(summary, (epoch+1))
            # Displaying the result after every n epochs 
            if (epoch + 1) % print_error_every == 0: 
                # Calculating the error 
                e = sess.run(error_, feed_dict = feed_dict) 
                print("Epoch", (epoch + 1), ": Error =", e, "m =", sess.run(m), "b =", sess.run(b)) 

                 
    return error_, parameters

### Experimento 1

epochs=10000  
lr=0.001

enfoque investigación científica: antes del experimento formular una hipótesis del comportamiento experado, ejecutar el experimento y utilizar los resultados de tensorboard para conculir.

In [55]:
Y = df_train['SalePrice']
X = df_train['OverallQual']

error_1, parametros_1 = run_training(X, Y, 10000, 0.001, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1301281800.0 m = 32954.117 b = -16686.4
Epoch 10000 : Error = 1243061000.0 m = 35580.223 b = -33504.598


### Experimento 2
Para este segundo experimento se mantendrá el valor del lr pero se modificará el número de iteraciones para ver observar si el valor del error disminuye notablemente.

epochs=15000  
lr=0.001

In [56]:
error_2, parametros_2 = run_training(X, Y, 20000, 0.001, 10000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 10000 : Error = 1243061000.0 m = 35580.223 b = -33504.598
Epoch 20000 : Error = 1184213800.0 m = 39288.344 b = -57252.31


### Experimento 3
Para este experimento se modifican ambos parámetros

epochs=10000  
lr=0.009

In [50]:
error_3, parametros_3 = run_training(X, Y, 10000, 0.009, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1150408800.0 m = 43538.13 b = -84468.836
Epoch 10000 : Error = 1146965200.0 m = 45189.527 b = -95044.81


Con este lr y cantidad de iteraciones se puede apreciar que el errordisminuyó notablemente.
A modo de comparación con los experimentos anteriores, se observa que desde la iteración 5000 el error obtenido ya era mucho menor que el de la iteracion 20000 del experimento 2, razón por la cual se decide mantener para el próximo experimento esta cantidad de iteraciones, e incrementar el lr.

### Experimento 4

Al aumenta el lr a 0.01 se espera que el error no disminuya mucho, pues solo se está incrementando un 0.001 en comparación con el experimento anterior, no obstante se realiza para apreciar la posibilidad de disminuir la cantidad de iteraciones. 

epochs=10000  
lr=0.01

In [51]:
error_4, parametros_4 = run_training(X, Y, 10000, 0.01, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1149091000.0 m = 43933.305 b = -86999.64
Epoch 10000 : Error = 1146935200.0 m = 45273.457 b = -95582.31


Como se esperaba no hubo mucho cambio en la disminución del error obtenido con respecto al experimento anterior, pero si se puede apreciar que el error no presenta mucha variación de las 5000 a las 10000 iteraciones, no obstante se decide seguir aumentando el lr y mantener la cantidad de iteraciones hasta que la disminución no sea notable.

### Experimento 5

epochs=10000  
lr=0.02

In [53]:
error_5, parametros_5 = run_training(X, Y, 10000, 0.02, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1146935400.0 m = 45273.63 b = -95583.41
Epoch 10000 : Error = 1146916600.0 m = 45410.695 b = -96461.234


Al obtener estos valores, se puede afirmar que el error no varió notablemente desde las 5000 iteraciones hasta las 10000, por lo que se decide bajar el nímero de iteraciones a 5000 para el próximo experimento.

### Experimento 6

epochs=5000  
lr=0.03

In [54]:
error_6, parametros_6 = run_training(X, Y, 5000, 0.03, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1146916500.0 m = 45399.062 b = -96386.72


Se decide seguir aumentando el lr, pues el valor del error no varió mucho con respecto al experimento anterior

### Experimento 7

Como no se ha observado un cambio importante en el error aumentando de 0.01 en 0.01, se decide incrementar aún más el lr

epochs=5000  
lr=0.09

In [60]:
error_7, parametros_7 = run_training(X, Y, 5000, 0.09, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = nan m = nan b = nan


Se aprecia que al lr ser tan alto, se obtienen valores nan en el error, por lo que se decide disminuir el mismo

### Experimento 8

epochs=5000  
lr=0.05

In [62]:
error_8, parametros_8 = run_training(X, Y, 5000, 0.05, 1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 1000 : Error = 25074963000000.0 m = -1061340.0 b = -259606.05
Epoch 2000 : Error = 3.4902491e+16 m = -41191908.0 b = -6534609.5
Epoch 3000 : Error = 4.858457e+19 m = -1538498700.0 b = -240335040.0
Epoch 4000 : Error = 6.7624435e+22 m = -57400040000.0 b = -8962804000.0
Epoch 5000 : Error = 9.413623e+25 m = -2141604000000.0 b = -334401370000.0


Este lr no lanza valores nan, pero sin embargo, se invirtieron los valores del error, y en vez de disminuir a partir de un punto comenzó a aumentar, por lo que este lr tampoco sirve, se continúa bajando su valor, pero esta vez muy poco.

### Experimento 9

epochs=5000  
lr=0.049

In [72]:
error_9, parametros_9 = run_training(X, Y, 5000, 0.049, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1146916100.0 m = 45411.77 b = -96468.09


Con esta disminución del lr no se obtienen nans en el valor de error, y se obtiene un error notablemente bajo con respecto a los experimentos anteriores, sin embargo, aunque ya se probó con anterioridad que si se aumenta el número de iteraciones el error no disminuye mucho, en aras de buscar el error mínimo se decide aumentar las iteraciones a 10000

### Experimento 10

epochs=10000  
lr=0.049

In [75]:
error_10, parametros_10 = run_training(X, Y, 10000, 0.049, 1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 1000 : Error = 1149296900.0 m = 43864.926 b = -86561.664
Epoch 2000 : Error = 1146939000.0 m = 45260.375 b = -95498.54
Epoch 3000 : Error = 1146916200.0 m = 45397.15 b = -96374.44
Epoch 4000 : Error = 1146916200.0 m = 45410.54 b = -96460.21
Epoch 5000 : Error = 1146916100.0 m = 45411.77 b = -96468.09
Epoch 6000 : Error = 1146916100.0 m = 45411.77 b = -96468.09
Epoch 7000 : Error = 1146916100.0 m = 45411.77 b = -96468.09
Epoch 8000 : Error = 1146916100.0 m = 45411.77 b = -96468.09
Epoch 9000 : Error = 1146916100.0 m = 45411.77 b = -96468.09
Epoch 10000 : Error = 1146916100.0 m = 45411.77 b = -96468.09


Se puede apreciar que con este lr y cantidad de iteraciones, despues de las 5000 iteraciones se mantiene el mismo error, sin embargo en aras de asegurar que no se puede conseguir un error más bajo, se decide aumentar el lr a 0.0499

### Experimento 11

epochs=5000  
lr=0.0499

In [69]:
error_11, parametros_11 = run_training(X, Y, 5000, 0.0499, 5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch 5000 : Error = 1543498000.0 m = 41016.195 b = -97155.23


El error con este último experimento aumentó.

### Conclusión

Se realizaron pruebas variando el valor de lr y de las iteraciones, concluyendo lo siguiente:

* El máximo número de lr que se puede utilizar es de 0.049, cualquier valor por encima (0.05, etc.) anulan el valor del error.
* Con lr por debajo de 0.009 habría que repetir más iteraciones para lograr el mismo resultado, lo cual conduciría a un mayor cómputo de datos.
* Con un lr de 0.049, despues de las 5000 iteraciones el número de error que se obtiene es el mismo.
* El mejor experimento fue el 9, con una cantidad de iteraciones de 5000 y un lr de 0.49, obteniendo el menor valor de error posible (1146916100.0).